In [37]:
import ccxt
import pandas as pd
import numpy as np

In [38]:
# Initialize Binance USDT-M Futures exchange
exchange = ccxt.binanceusdm()

exchange

ccxt.binanceusdm()

In [59]:
exchange.liq

AuthenticationError: binanceusdm requires "apiKey" credential

In [48]:
markets = exchange.fetch_markets()

# Filter symbols paired to USDT
usdt_symbols = [
    market["symbol"][:-5]
    for market in markets
    if market["quote"] == "USDT"
]

# Select top 200 symbols
symbols = usdt_symbols[:200]

' '.join(symbols)

'BTC/USDT ETH/USDT BCH/USDT XRP/USDT EOS/USDT LTC/USDT TRX/USDT ETC/USDT LINK/USDT XLM/USDT ADA/USDT XMR/USDT DASH/USDT ZEC/USDT XTZ/USDT BNB/USDT ATOM/USDT ONT/USDT IOTA/USDT BAT/USDT VET/USDT NEO/USDT QTUM/USDT IOST/USDT THETA/USDT ALGO/USDT ZIL/USDT KNC/USDT ZRX/USDT COMP/USDT OMG/USDT DOGE/USDT SXP/USDT KAVA/USDT BAND/USDT RLC/USDT WAVES/USDT MKR/USDT SNX/USDT DOT/USDT DEFI/USDT YFI/USDT BAL/USDT CRV/USDT TRB/USDT RUNE/USDT SUSHI/USDT EGLD/USDT SOL/USDT ICX/USDT STORJ/USDT BLZ/USDT UNI/USDT AVAX/USDT FTM/USDT ENJ/USDT FLM/USDT REN/USDT KSM/USDT NEAR/USDT AAVE/USDT FIL/USDT RSR/USDT LRC/USDT OCEAN/USDT CVC/USDT BEL/USDT CTK/USDT AXS/USDT ALPHA/USDT ZEN/USDT SKL/USDT GRT/USDT 1INCH/USDT CHZ/USDT SAND/USDT ANKR/USDT LIT/USDT UNFI/USDT REEF/USDT RVN/USDT SFP/USDT XEM/USDT BTCST/USDT COTI/USDT CHR/USDT MANA/USDT ALICE/USDT HBAR/USDT ONE/USDT LINA/USDT STMX/USDT DENT/USDT CELR/USDT HOT/USDT MTL/USDT OGN/USDT NKN/USDT SC/USDT DGB/USDT 1000SHIB/USDT BAKE/USDT GTC/USDT BTCDOM/USDT IOTX/USDT

In [54]:
len(markets)

393

In [58]:
market_df = pd.DataFrame(markets)
market_df.head()
market_df['info'][0]

{'symbol': 'BTCUSDT',
 'pair': 'BTCUSDT',
 'contractType': 'PERPETUAL',
 'deliveryDate': 4133404800000,
 'onboardDate': 1569398400000,
 'status': 'TRADING',
 'maintMarginPercent': '2.5000',
 'requiredMarginPercent': '5.0000',
 'baseAsset': 'BTC',
 'quoteAsset': 'USDT',
 'marginAsset': 'USDT',
 'pricePrecision': 2,
 'quantityPrecision': 3,
 'baseAssetPrecision': 8,
 'quotePrecision': 8,
 'underlyingType': 'COIN',
 'underlyingSubType': ['PoW'],
 'triggerProtect': '0.0500',
 'liquidationFee': '0.012500',
 'marketTakeBound': '0.05',
 'maxMoveOrderLimit': 10000,
 'filters': [{'maxPrice': '4529764',
   'filterType': 'PRICE_FILTER',
   'minPrice': '556.80',
   'tickSize': '0.10'},
  {'maxQty': '1000',
   'stepSize': '0.001',
   'filterType': 'LOT_SIZE',
   'minQty': '0.001'},
  {'stepSize': '0.001',
   'filterType': 'MARKET_LOT_SIZE',
   'maxQty': '120',
   'minQty': '0.001'},
  {'filterType': 'MAX_NUM_ORDERS', 'limit': 200},
  {'filterType': 'MAX_NUM_ALGO_ORDERS', 'limit': 10},
  {'notional'

In [40]:
def fetch_ohlcv(symbol, since, timeframe="1d"):
    """
    Fetch historical OHLCV data for a given symbol.
    """

    try:
        ohlcv = exchange.fetch_ohlcv(
            symbol, timeframe=timeframe, since=since, limit=1000
        )
        df = pd.DataFrame(
            ohlcv, columns=["timestamp", "open", "high", "low", "close", "volume"]
        )
        df["datetime"] = pd.to_datetime(df["timestamp"], unit="ms")
        df.set_index("datetime", inplace=True)
        return df["close"]
    except Exception as e:
        print(f"Could not fetch data for {symbol}: {e}")
        return None

In [41]:
def filter_and_save_correlations(correlations, threshold, output_file):
    """
    Filter symbols based on correlation threshold and save to a text file.
    """
    # Filter symbols with correlation higher than the threshold
    filtered_symbols = [
        symbol for symbol, corr in correlations.items() if corr > threshold
    ]
    # Format symbols to match the required text format
    formatted_symbols = ",".join(
        [
            f"BINANCE:{symbol.replace('/', '').replace('1000', '1000').replace('1M', '1M')}.P"
            for symbol in filtered_symbols
        ]
    )
    # Save to text file
    with open(output_file, "w") as file:
        file.write(formatted_symbols)
    print(f"{len(filtered_symbols)} filtered symbols saved to {output_file}")

In [42]:
# Fetch historical prices starting from January 1, 2023
price_data = {}
since = exchange.parse8601("2023-01-01T00:00:00Z")

for symbol in symbols:
    print(f"Fetching data for {symbol}")
    data = fetch_ohlcv(symbol, since)
    if data is not None:
        price_data[symbol] = data

Fetching data for BTC/USDT
Fetching data for ETH/USDT
Fetching data for BCH/USDT
Fetching data for XRP/USDT
Fetching data for EOS/USDT
Fetching data for LTC/USDT
Fetching data for TRX/USDT
Fetching data for ETC/USDT
Fetching data for LINK/USDT
Fetching data for XLM/USDT
Fetching data for ADA/USDT
Fetching data for XMR/USDT
Fetching data for DASH/USDT
Fetching data for ZEC/USDT
Fetching data for XTZ/USDT
Fetching data for BNB/USDT
Fetching data for ATOM/USDT
Fetching data for ONT/USDT
Fetching data for IOTA/USDT
Fetching data for BAT/USDT
Fetching data for VET/USDT
Fetching data for NEO/USDT
Fetching data for QTUM/USDT
Fetching data for IOST/USDT
Fetching data for THETA/USDT
Fetching data for ALGO/USDT
Fetching data for ZIL/USDT
Fetching data for KNC/USDT
Fetching data for ZRX/USDT
Fetching data for COMP/USDT
Fetching data for OMG/USDT
Fetching data for DOGE/USDT
Fetching data for SXP/USDT
Fetching data for KAVA/USDT
Fetching data for BAND/USDT
Fetching data for RLC/USDT
Fetching data f

In [43]:
# Create a DataFrame of prices and calculate daily returns
prices_df = pd.DataFrame(price_data)
prices_df = prices_df.dropna(axis=1, how="any")  # Remove symbols with insufficient data
returns_df = prices_df.pct_change().dropna()

returns_df

,BTC/USDT,ETH/USDT,BCH/USDT,XRP/USDT,EOS/USDT,LTC/USDT,TRX/USDT,ETC/USDT,LINK/USDT,XLM/USDT,...,INJ/USDT,STG/USDT,SPELL/USDT,1000LUNC/USDT,LUNA2/USDT,LDO/USDT,CVX/USDT,ICP/USDT,APT/USDT,QNT/USDT
datetime,,,,,,,,,,,,,,,,,,,,,
2023-01-02,0.003353,0.011059,0.025527,0.027770,0.003413,0.053753,0.007306,0.020911,0.011198,0.017106,...,0.020440,0.016294,0.007509,0.061211,0.030428,0.116546,0.004057,-0.001245,0.067586,0.027078
2023-01-03,0.000072,0.000824,0.003930,-0.012935,-0.002268,0.011246,-0.006890,-0.014214,-0.012480,-0.002034,...,0.004622,-0.001902,0.000186,-0.022571,-0.007420,-0.003485,-0.012123,-0.024925,0.016372,-0.010324
2023-01-04,0.010494,0.034722,0.018571,0.010483,0.030682,-0.001456,-0.000913,0.207719,0.033286,0.002718,...,0.038344,0.002450,0.019188,0.014112,0.013578,0.147794,0.004720,0.007669,-0.002226,0.041822
2023-01-05,-0.001087,-0.004226,-0.003154,-0.026801,-0.017641,-0.016043,-0.019371,-0.049945,-0.025495,-0.012063,...,0.002954,-0.023357,-0.006032,-0.038583,-0.020245,-0.028727,-0.031005,-0.032217,-0.026619,-0.010460
2023-01-06,0.007133,0.013956,0.004449,0.018656,0.004489,0.022638,-0.022736,0.120288,0.009369,0.028262,...,0.010309,-0.016129,0.014527,0.026316,0.017284,0.133468,-0.002909,0.021494,0.040319,0.010029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-14,0.000090,-0.009474,-0.015747,-0.009207,-0.060415,-0.050606,-0.028801,-0.043128,0.004623,-0.031228,...,-0.061825,-0.048449,-0.049262,-0.035269,-0.058164,0.002991,0.000000,-0.048130,-0.056722,-0.025950
2024-12-15,0.030375,0.023238,0.023211,0.019836,0.037428,0.024960,0.011763,0.037729,0.004808,0.014926,...,0.041624,0.026602,0.031493,0.038649,0.027344,0.076085,0.000000,0.025010,0.018546,0.011737
2024-12-16,0.014961,0.006634,-0.014629,0.015282,-0.040703,-0.025920,0.038941,-0.016935,-0.018013,-0.026835,...,-0.003289,-0.032600,-0.025786,-0.024310,-0.029875,-0.062266,0.000000,-0.039024,0.019028,-0.052879


In [44]:
# Calculate correlation of each coin's returns with BTC
btc_returns = returns_df["BTC/USDT"]
correlations = {}
for symbol in returns_df.columns:
    corr = returns_df[symbol].corr(btc_returns)
    correlations[symbol] = corr
        
correlations

/opt/homebrew/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/homebrew/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


{'BTC/USDT': 0.9999999999999999,
 'ETH/USDT': 0.8051812784273454,
 'BCH/USDT': 0.5581896035087389,
 'XRP/USDT': 0.4089783216187576,
 'EOS/USDT': 0.5834271329128334,
 'LTC/USDT': 0.5674050280067494,
 'TRX/USDT': 0.22842220549184825,
 'ETC/USDT': 0.6599890251995119,
 'LINK/USDT': 0.576832483989959,
 'XLM/USDT': 0.3767514004000797,
 'ADA/USDT': 0.6545496757044643,
 'XMR/USDT': 0.36953220936849923,
 'DASH/USDT': 0.5880773642016611,
 'ZEC/USDT': 0.511414776365399,
 'XTZ/USDT': 0.5882938620469428,
 'BNB/USDT': 0.6308339998812164,
 'ATOM/USDT': 0.6180319940667864,
 'ONT/USDT': 0.5936724095074839,
 'IOTA/USDT': 0.5500439769039122,
 'BAT/USDT': 0.6728731070526153,
 'VET/USDT': 0.6232945043942408,
 'NEO/USDT': 0.5816401718286586,
 'QTUM/USDT': 0.6368750300292743,
 'IOST/USDT': 0.6641140019342452,
 'THETA/USDT': 0.6470880911534718,
 'ALGO/USDT': 0.6007675399732035,
 'ZIL/USDT': 0.6578384955635689,
 'KNC/USDT': 0.5939622419149482,
 'ZRX/USDT': 0.4855031680305339,
 'COMP/USDT': 0.5452030161918626,


In [45]:
# Sort correlations in descending order
sorted_correlations = dict(
    sorted(correlations.items(), key=lambda item: item[1], reverse=True)
)

# Print correlations
print("\nCorrelation of each coin's returns with BTC:")
for symbol, corr in sorted_correlations.items():
    print(f"{symbol}: {corr:.4f}")


Correlation of each coin's returns with BTC:
BTC/USDT: 1.0000
ETH/USDT: 0.8052
DEFI/USDT: 0.7337
BAL/USDT: 0.7215
DOGE/USDT: 0.7084
DENT/USDT: 0.6744
BAT/USDT: 0.6729
ONE/USDT: 0.6667
SOL/USDT: 0.6658
IOST/USDT: 0.6641
ETC/USDT: 0.6600
ZIL/USDT: 0.6578
RUNE/USDT: 0.6555
ADA/USDT: 0.6545
THETA/USDT: 0.6471
AVAX/USDT: 0.6447
DOT/USDT: 0.6417
QTUM/USDT: 0.6369
1INCH/USDT: 0.6338
LIT/USDT: 0.6309
BNB/USDT: 0.6308
ENJ/USDT: 0.6260
ANKR/USDT: 0.6254
VET/USDT: 0.6233
LRC/USDT: 0.6231
CELR/USDT: 0.6189
HOT/USDT: 0.6189
ATOM/USDT: 0.6180
FIL/USDT: 0.6174
MANA/USDT: 0.6160
KAVA/USDT: 0.6150
NKN/USDT: 0.6149
ICX/USDT: 0.6145
NEAR/USDT: 0.6137
RVN/USDT: 0.6033
ALGO/USDT: 0.6008
FTM/USDT: 0.5986
KNC/USDT: 0.5940
ONT/USDT: 0.5937
XTZ/USDT: 0.5883
DASH/USDT: 0.5881
ALICE/USDT: 0.5837
EOS/USDT: 0.5834
NEO/USDT: 0.5816
CHZ/USDT: 0.5811
LINK/USDT: 0.5768
BAND/USDT: 0.5749
LTC/USDT: 0.5674
ALPHA/USDT: 0.5672
EGLD/USDT: 0.5664
BCH/USDT: 0.5582
IOTA/USDT: 0.5500
COMP/USDT: 0.5452
SXP/USDT: 0.5279
OMG/USDT

In [46]:
# Set correlation threshold and output file name
threshold = 0.6
output_file = "HighCorrelation.txt"

# Filter and save symbols with high correlation
filter_and_save_correlations(sorted_correlations, threshold, output_file)

Filtered symbols saved to HighCorrelation.txt
